# $match 的操作

In [ ]:
# $match all celestial bodies, not equal to Star
db.solarSystem.aggregate([
    {
        $match: {
            type: { $ne: "Star" }
        }
    }
]).pretty()

In [ ]:
# same query using find command
db.solarSystem.find({ type: { $ne: "Star" }}).pretty()

In [ ]:
# count the number of matching documents
db.solarSystem.find({ type: { $ne: "Star"}}).count()

In [ ]:
# using $count
db.solarSystem.aggregate([
    {
        $match: {
            type: { $ne: "Star"}
        }
    },
    { $count: "planets"}
])

In [ ]:
# matching on value, and removing ``_id`` from projected document
db.solarSystem.find({ name: "Earth" }, { "_id": 0 }).pretty()

### 範例01

In [ ]:
var pipeline = [
    {
        $match: {
            "imdb.rating": { $gte: 7 },
            genres: { $nin: [ "Crime", "Horror" ]},
            rated: { $in: [ "PG", "G" ]},
            languages: { $all: [ "English", "Japanese" ]}
        }
    }
]

db.movies.aggregate(pipeline).pretty()

# $project 的操作

In [ ]:
# project ``name`` and remove ``_id``
db.solarSystem.aggregate([
    {
        $project: {
            _id: 0,
            name:1
        }
    }
]).pretty()

In [ ]:
# project ``name`` and ``gravity`` fields, including default ``_id``
db.solarSystem.aggregate([
    {
        $project: {
            name: 1,
            gravity: 1
        }
    }
]).pretty()

In [ ]:
# using dot-notation to express the projection fields
db.solarSystem.aggregate([
    {
        $project: {
            _id: 0,
            name: 1,
            "gravity.value": 1
        }
    }
])

In [ ]:
# reassing ``gravity`` field with value from ``gravity.value`` embeded field
db.solarSystem.aggregate([
    {
        $project: {
            _id: 0,
            name: 1,
            gravity: "$gravity.value"
        }
    }
])

In [ ]:
# creating a document new field ``surfaceGravity``
db.solarSystem.aggregate([
    {
        $project: {
            _id: 0,
            name: 1,
            surfaceGravity: "$gravity.value"
        }
    }
])

In [ ]:
# creating a new field ``myWeight`` using expressions
db.solarSystem.aggregate([
    {
        $project: {
            _id: 0,
            name: 1,
            myWeight: { $multiply: [ { $divide: [ "$gravity.value", 9.8 ] }, 72 ] }
        }
    }
]).pretty()

### 範例02

In [ ]:
var pipeline = [
    {
        $match: {
            "imdb.rating": { $gte: 7 },
            genres: { $nin: [ "Crime", "Horror" ]},
            rated: { $in: [ "PG", "G" ]},
            languages: { $all: [ "English", "Japanese" ]}
        }
    },
    {
        $project: {
            _id: 0,
            title: 1,
            rated: 1
        }
    }
]

db.movies.aggregate(pipeline).pretty()

### 範例03

In [ ]:
# 參考01
# 應該先要考慮到 title 的直是不是字串
db.movies.aggregate([
    {
        $project:{
            titleSplit: { $split: ["$title", " "] },
        }
    },
    {
        $match: {
            titleSplit: { $size: 1 }
        }
    }
]).itcount()

In [ ]:
# 參考02
db.movies.aggregate([
  {
    $match: {
      title: {
        $type: "string"
      }
    }
  },
  {
    $project: {
      title: { $split: ["$title", " "] },
      _id: 0
    }
  },
  {
    $match: {
      title: { $size: 1 }
    }
  }
]).itcount()

### 範例04

In [ ]:
# 參考01
db.movies.aggregate([
    {
        $match: {
            cast: { $elemMatch: { $exists: true}},
            directors: { $elemMatch: { $exists: true}},
            writers: { $elemMatch: { $exists: true}}
        }
    },
    {
        $project: {
            _id: 0,
            cast: 1,
            directors: 1,
            writers: { $map: {
                input: "$writers",
                as: "writer",
                in: { $arrayElemAt: [{ $split: [ "$$writer", " (" ]}, 0 ]}
            }}
        }
    },
    {
        $project: {
            "samePerson": { $setIntersection: [ "$cast", "$directors", "$writers" ]}
        }
    },
    {
        $match: {
            "samePerson": { $elemMatch: { $exists: true }}
        }
    }
]).itcount()

In [ ]:
# 參考02
db.movies.aggregate([
  {
    $match: {
      cast: { $elemMatch: { $exists: true } },
      directors: { $elemMatch: { $exists: true } },
      writers: { $elemMatch: { $exists: true } }
    }
  },
  {
    $project: {
      _id: 0,
      cast: 1,
      directors: 1,
      writers: {
        $map: {
          input: "$writers",
          as: "writer",
          in: {
            $arrayElemAt: [
              {
                $split: ["$$writer", " ("]
              },
              0
            ]
          }
        }
      }
    }
  },
  {
    $project: {
      labor_of_love: {
        $gt: [
          { $size: { $setIntersection: ["$cast", "$directors", "$writers"] } },
          0
        ]
      }
    }
  },
  {
    $match: { labor_of_love: true }
  },
  {
    $count: "labors of love"
  }
])